In [15]:
!pip install pandas mysql-connector

In [5]:
import zipfile
import pandas as pd

# Define the ZIP file path
zip_file_path = "household_power_consumption.zip"

# Extract the file
with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall("unzipped_data")  # Extract files to a new folder

# Get the extracted file path
extracted_file_path = "unzipped_data/household_power_consumption.txt"

# Load dataset correctly
df = pd.read_csv(extracted_file_path, delimiter=";", low_memory=False)

# Display first few rows
print(df.head())

         Date      Time Global_active_power Global_reactive_power  Voltage  \
0  16/12/2006  17:24:00               4.216                 0.418  234.840   
1  16/12/2006  17:25:00               5.360                 0.436  233.630   
2  16/12/2006  17:26:00               5.374                 0.498  233.290   
3  16/12/2006  17:27:00               5.388                 0.502  233.740   
4  16/12/2006  17:28:00               3.666                 0.528  235.680   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0           18.400          0.000          1.000            17.0  
1           23.000          0.000          1.000            16.0  
2           23.000          0.000          2.000            17.0  
3           23.000          0.000          1.000            17.0  
4           15.800          0.000          1.000            17.0  


In [39]:
import pandas as pd

# Load dataset with optimal settings
df = pd.read_csv("unzipped_data/household_power_consumption.txt", delimiter=";", low_memory=False)
print(df.head())  # Check the first few rows

         Date      Time Global_active_power Global_reactive_power  Voltage  \
0  16/12/2006  17:24:00               4.216                 0.418  234.840   
1  16/12/2006  17:25:00               5.360                 0.436  233.630   
2  16/12/2006  17:26:00               5.374                 0.498  233.290   
3  16/12/2006  17:27:00               5.388                 0.502  233.740   
4  16/12/2006  17:28:00               3.666                 0.528  235.680   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0           18.400          0.000          1.000            17.0  
1           23.000          0.000          1.000            16.0  
2           23.000          0.000          2.000            17.0  
3           23.000          0.000          1.000            17.0  
4           15.800          0.000          1.000            17.0  


In [41]:
df["Datetime"] = pd.to_datetime(df["Date"] + " " + df["Time"], format="%d/%m/%Y %H:%M:%S")
df.drop(["Date", "Time"], axis=1, inplace=True)
print(df.head())

  Global_active_power Global_reactive_power  Voltage Global_intensity  \
0               4.216                 0.418  234.840           18.400   
1               5.360                 0.436  233.630           23.000   
2               5.374                 0.498  233.290           23.000   
3               5.388                 0.502  233.740           23.000   
4               3.666                 0.528  235.680           15.800   

  Sub_metering_1 Sub_metering_2  Sub_metering_3            Datetime  
0          0.000          1.000            17.0 2006-12-16 17:24:00  
1          0.000          1.000            16.0 2006-12-16 17:25:00  
2          0.000          2.000            17.0 2006-12-16 17:26:00  
3          0.000          1.000            17.0 2006-12-16 17:27:00  
4          0.000          1.000            17.0 2006-12-16 17:28:00  


In [5]:
print(df.isnull().sum())  # Shows how many missing values are in each column

Date                         0
Time                         0
Global_active_power          0
Global_reactive_power        0
Voltage                      0
Global_intensity             0
Sub_metering_1               0
Sub_metering_2               0
Sub_metering_3           25979
dtype: int64


In [9]:
df["Sub_metering_3"] = df["Sub_metering_3"].fillna(df["Sub_metering_3"].mean())

In [33]:
print(df.isnull().sum())

Date                     0
Time                     0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64


In [ ]:
df["Datetime"] = pd.to_datetime(df["Date"] + " " + df["Time"])  # Merge date & time
df.drop(columns=["Date", "Time"], inplace=True)  # Remove old columns
df.to_csv("cleaned_household_power.csv", index=False)  # Overwrite with correct format

In [ ]:
df = pd.read_csv("cleaned_household_power.csv")  # Ensure only 8 columns
print(df.columns)  # Check if 'Date' and 'Time' are gone

In [169]:
df = df[df["Datetime"].astype(str).str.match(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}")]

In [171]:
df["Datetime"] = pd.to_datetime(df["Datetime"], errors="coerce")

In [175]:
df.to_csv("cleaned_household_power.csv", index=False)  # Save cleaned data

In [179]:
print(df["Datetime"].isna().sum())  # Count missing values

0


In [ ]:
df.dropna(subset=["Datetime"], inplace=True)  # Remove rows with missing timestamps

In [ ]:
df["Datetime"].fillna(method="ffill", inplace=True)  # Fill using previous valid date

In [73]:
conn = sqlite3.connect("power_pulse.db")

# Reinsert correctly structured data
df.to_sql("power_consumption", conn, if_exists="replace", index=False)

conn.close()
print("✅ Data successfully reinserted with correct column mapping!")

✅ Data successfully reinserted with correct column mapping!


In [7]:
df.to_csv("cleaned_household_power.csv", index=False)  # Save cleaned data

In [42]:
df.columns

Index(['Global_active_power', 'Global_reactive_power', 'Voltage',
       'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3', 'Datetime'],
      dtype='object')

In [ ]:
print(df["Datetime"].isna().sum())  # Count NaT entries
print(df.index.isna().sum())  # Check if index has NaT values

In [19]:
df.dropna(subset=["Datetime"], inplace=True)
df.set_index("Datetime", inplace=True)  # Reset index after dropping

In [71]:
df = df[["Datetime", "Global_active_power", "Global_reactive_power", "Voltage",
         "Global_intensity", "Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]]

In [65]:
import sqlite3

# Connect to SQLite database (creates if it doesn't exist)
conn = sqlite3.connect("power_pulse.db")
cursor = conn.cursor()

# Create the table for power consumption
cursor.execute("""
    CREATE TABLE IF NOT EXISTS power_consumption (
        Datetime TEXT,
        Global_active_power REAL,
        Global_reactive_power REAL,
        Voltage REAL,
        Global_intensity REAL,
        Sub_metering_1 REAL,
        Sub_metering_2 REAL,
        Sub_metering_3 REAL
    )
""")

conn.commit()
print("Table created successfully!")

conn.close()

Table created successfully!


In [151]:
conn = sqlite3.connect("power_pulse.db")
cursor = conn.cursor()

# Ensure we're passing only 8 values per row
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO power_consumption VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, tuple(row))  # Now guaranteed to be only 8 values

conn.commit()
conn.close()
print("Data inserted successfully!")

Data inserted successfully!


In [61]:
import sqlite3

conn = sqlite3.connect("power_pulse.db")
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM power_consumption")
row_count = cursor.fetchone()[0]

conn.close()
print(f"Total Rows in Database: {row_count}")

Total Rows in Database: 2075259


In [1]:
%%writefile Project_3.py
import streamlit as st
import sqlite3
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# ---- DATABASE CONNECTION ----
@st.cache_data
def load_data():
    conn = sqlite3.connect("power_pulse.db")
    query = "SELECT * FROM power_consumption ORDER BY Datetime DESC LIMIT 10000"
    df = pd.read_sql(query, conn)
    conn.close()

    # Convert 'Datetime' column properly
    df["Datetime"] = pd.to_datetime(df["Datetime"], errors="coerce")

    # Convert numeric columns for proper aggregation
    numeric_cols = ["Global_active_power", "Global_reactive_power", "Voltage",
                    "Global_intensity", "Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]

    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")  # Convert to numeric safely

    # Aggregate to hourly intervals
    df = df.set_index("Datetime").resample("H")[numeric_cols].mean().reset_index()

    return df

df = load_data()

# ---- NAVIGATION ----
page = st.sidebar.selectbox("Select Page", ["Dashboard", "Project Summary"])

# ---- PROJECT SUMMARY PAGE ----
if page == "Project Summary":
    st.title("📌 Project Summary")
    
    st.write("### 🔹 Project Name: PowerPulse - Household Energy Usage Forecast")
    st.write("This project aims to analyze household power consumption data and build predictive models to optimize energy usage.")
    
    st.write("### 🔹 Key Objectives")
    st.markdown("- Perform **Exploratory Data Analysis (EDA)** to uncover patterns in energy consumption.")
    st.markdown("- Develop **feature engineering techniques** to improve prediction accuracy.")
    st.markdown("- Build and evaluate **predictive models** using regression and time-series analysis.")
    
    st.write("### 🔹 Data Overview")
    st.write("The dataset contains power consumption metrics, including active power, voltage fluctuations, and appliance-level usage.")

    st.write("### 🔹 Evaluation Metrics")
    st.markdown("- **Root Mean Squared Error (RMSE):** Evaluates prediction errors.")
    st.markdown("- **Mean Absolute Error (MAE):** Measures the average magnitude of prediction errors.")
    st.markdown("- **R² Score:** Assesses how well the model explains variations in power consumption.")

    st.write("### 🔹 Conclusion")
    st.write("This project provides a **data-driven approach** to forecasting energy consumption, enabling users to optimize power usage and reduce costs.")

# ---- DASHBOARD PAGE ----
if page == "Dashboard":
    st.title("⚡ Household Power Consumption Dashboard")
    st.write("Explore trends, patterns, and distribution in power usage data.")

    # ---- DATE FILTER ----
    st.sidebar.write("## Filter Data")
    start_date = st.sidebar.date_input("Start Date", df["Datetime"].min().date())
    end_date = st.sidebar.date_input("End Date", df["Datetime"].max().date())

    # Filter data based on user input
    filtered_df = df[(df["Datetime"] >= str(start_date)) & (df["Datetime"] <= str(end_date))]

    # ---- DISPLAY FILTERED DATA ----
    if not filtered_df.empty:
        st.write("### Filtered Data View")
        st.dataframe(filtered_df)
    else:
        st.warning("No data available for the selected date range. Adjust your filter.")

    # ---- VISUALIZATIONS ----
    st.write("## 🔹 Power Consumption Trends")
    if not filtered_df.empty:
        st.line_chart(filtered_df[["Datetime", "Global_active_power"]], x="Datetime", y="Global_active_power")
    else:
        st.warning("No data available for visualization.")

    st.write("## 🔹 Voltage Distribution")
    if not filtered_df.empty:
        st.bar_chart(filtered_df["Voltage"])
    else:
        st.warning("No data available for visualization.")

    st.write("## 🔹 Sub-Metering Usage Over Time")
    if not filtered_df.empty:
        st.line_chart(filtered_df[["Datetime", "Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]], x="Datetime")
    else:
        st.warning("No data available for visualization.")

    # ---- FEATURE IMPORTANCE ANALYSIS ----
    st.write("## 🔹 Feature Importance - Correlation Heatmap")
    if not filtered_df.empty:
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.heatmap(filtered_df.drop(columns=["Datetime"]).corr(), annot=True, cmap="coolwarm", ax=ax)
        st.pyplot(fig)
    else:
        st.warning("No data available for feature correlation analysis.")

    # ---- MODEL PERFORMANCE ----
    st.write("## 🔹 Model Evaluation Metrics")
    if not filtered_df.empty:
        # Placeholder values (Replace with actual calculated metrics)
        st.write("**RMSE:** 0.245")
        st.write("**MAE:** 0.180")
        st.write("**R² Score:** 0.92")
    else:
        st.warning("No data available for model evaluation.")

    # ---- ACTUAL VS PREDICTED VISUALIZATION ----
    st.write("## 🔹 Predicted vs. Actual Power Consumption")
    if not filtered_df.empty:
        # Placeholder prediction logic (Replace with actual model predictions)
        filtered_df["Predicted"] = filtered_df["Global_active_power"] * 1.02  # Example adjustment
        st.line_chart(filtered_df[["Datetime", "Global_active_power", "Predicted"]], x="Datetime")
    else:
        st.warning("No data available for predictions.")

Overwriting Project_3.py
